In [1]:
import pandas as pd
import numpy as np
import scipy as sp

import keras
from keras import backend as K
from keras.datasets import mnist
from keras.models import Sequential, Model, load_model
from keras.layers import Flatten, Dense, Dropout, Lambda, Activation
from keras.optimizers import SGD, Adam
from keras.losses import categorical_crossentropy
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load mnist data & normalize to 0-1
# 1D data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28*28).astype('float32')
x_test = x_test.reshape(-1, 28*28).astype('float32')
x_train /= 255.
x_test /= 255. 

# 2D data
(x_train_2D, _), (x_test_2D, _) = mnist.load_data()
x_train_2D = x_train_2D.reshape(x_train_2D.shape[0], 1, 28, 28).astype('float32')
x_test_2D = x_test_2D.reshape(x_test_2D.shape[0], 1, 28, 28).astype('float32')
x_train_2D /= 255.
x_test_2D /= 255.

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 784) (60000, 10) (10000, 784) (10000, 10)


### Teacher Net 
1. DENSE 1200-1200-10 with HEAVY REGULARIZATION
2. CNN

In [3]:
k_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)
k_constraint = keras.constraints.MaxNorm(max_value=15, axis=0)

mnist_dense = Sequential()
mnist_dense.add(Dense(1200, name='hidden_1', input_shape=(28*28, ), activation='relu', kernel_initializer=k_init, kernel_constraint=k_constraint))
mnist_dense.add(Dropout(0.7, name='dropout_1'))
mnist_dense.add(Dense(1200, name='hidden_2', activation='relu', kernel_initializer=k_init, kernel_constraint=k_constraint))
mnist_dense.add(Dropout(0.7, name='dropout_2'))
mnist_dense.add(Dense(10, name='logit'))
mnist_dense.add(Activation('softmax', name='softmax'))

mnist_dense.compile(loss=categorical_crossentropy, optimizer=Adam(0.0001), metrics=['accuracy'])

mnist_dense.fit(x_train, y_train, batch_size=100, epochs=20, verbose=1, validation_data=(x_test, y_test))

loss, accuracy = mnist_dense.evaluate(x_test, y_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('DENSE TECHERT NET - On test set:')
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))

mnist_dense.save('./models/mnist_teacher_dense.h5')

In [4]:
mnist_dense = load_model('./models/mnist_teacher_dense.h5')
loss, accuracy = mnist_dense.evaluate(x_test, y_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('DENSE TECHERT NET - On test set:')
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))

DENSE TECHERT NET - On test set:
loss = 0.055868843965092674, accuracy = 0.9832, #errors = 168


In [5]:
# fit a CNN teacher net
mnist_cnn = Sequential()

mnist_cnn.add(Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=(1, 28, 28), name='conv_1'))
mnist_cnn.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_2'))
mnist_cnn.add(MaxPooling2D(pool_size=(2, 2), padding='same', name='pool_1'))
mnist_cnn.add(Dropout(0.25, name='dropout_1'))

mnist_cnn.add(Flatten())
mnist_cnn.add(Dense(10, name='logit'))
mnist_cnn.add(Activation('softmax', name='softmax'))

mnist_cnn.compile(loss=categorical_crossentropy, optimizer=Adam(lr=0.0005), metrics=['accuracy'])
mnist_cnn.fit(x_train_2D, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test_2D, y_test))

mnist_cnn.save('./models/mnist_teacher_cnn.h5')

In [6]:
mnist_cnn = load_model('./models/mnist_teacher_cnn.h5')
loss, accuracy = mnist_cnn.evaluate(x_test_2D, y_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('CNN TEACHER NET - On test set:')
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))
print()

CNN TEACHER NET - On test set:
loss = 0.030440742732951186, accuracy = 0.9903, #errors = 97



### Student Net 1
- NO DISTILLATION BASELINE

In [7]:
def MNIST_StudentNet(n_hidden, T):
    '''
    Function to build a studnet net
    '''
    model = Sequential()
    model.add(Dense(n_hidden, name='hidden_1', input_shape=(28*28, ), activation='relu'))
    model.add(Dense(n_hidden, name='hidden_2', activation='relu'))
    model.add(Dense(10, name='logit'))
    model.add(Lambda(lambda x: x / T, name='logit_soft'))
    model.add(Activation('softmax', name='softmax'))
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
    return model

In [8]:
# train baseline student net - NO distillation
mnist_student_basline = MNIST_StudentNet(n_hidden=20, T=1)
mnist_student_basline.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
mnist_student_basline.fit(x_train, y_train, batch_size=128, epochs=50, verbose=1, validation_data=(x_test, y_test))

# baseline student net model evaluation
loss, accuracy = mnist_student_basline.evaluate(x_test, y_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('STUDENT BASELINE - On test set:')
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 3s 44us/step - loss: 0.5879 - acc: 0.8240 - val_loss: 0.2767 - val_acc: 0.9236
Epoch 2/50
60000/60000 [==============================] - 1s 23us/step - loss: 0.2539 - acc: 0.9275 - val_loss: 0.2207 - val_acc: 0.9364
Epoch 3/50
60000/60000 [==============================] - 1s 22us/step - loss: 0.2081 - acc: 0.9404 - val_loss: 0.2061 - val_acc: 0.9383
Epoch 4/50
60000/60000 [==============================] - 1s 25us/step - loss: 0.1822 - acc: 0.9475 - val_loss: 0.1834 - val_acc: 0.9473
Epoch 5/50
60000/60000 [==============================] - 1s 25us/step - loss: 0.1647 - acc: 0.9527 - val_loss: 0.1707 - val_acc: 0.9512
Epoch 6/50
60000/60000 [==============================] - 1s 23us/step - loss: 0.1513 - acc: 0.9568 - val_loss: 0.1581 - val_acc: 0.9537
Epoch 7/50
60000/60000 [==============================] - 1s 23us/step - loss: 0.1416 - acc: 0.9590 - val_loss: 0.1510 - val_acc

### Student Net 2
- DISTILLED --- DENSE TEACHER

In [9]:
def get_layer_output(model, layer_name):
    output = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    return output

# compute 'soft target'
T = 3
teacher_logit = get_layer_output(mnist_dense, 'logit')
logit_train = teacher_logit.predict(x_train)
y_train_soft = K.softmax(logit_train / T).eval(session=K.get_session())

# train student net distilled from the dense teacher net
mnist_student_distilled = MNIST_StudentNet(n_hidden=20, T=T)
mnist_student_distilled.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
mnist_student_distilled.fit(x_train, y_train_soft, 
                            batch_size=128, epochs=50, verbose=1, validation_data=(x_test, y_test))

# distilled student net model evaluation
loss, accuracy = mnist_student_distilled.evaluate(x_test, y_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('DISTILLED STUDENT - On test set:')
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 2s 39us/step - loss: 1.0700 - acc: 0.7853 - val_loss: 0.4012 - val_acc: 0.9120
Epoch 2/50
60000/60000 [==============================] - 2s 31us/step - loss: 0.7297 - acc: 0.9250 - val_loss: 0.3376 - val_acc: 0.9288
Epoch 3/50
60000/60000 [==============================] - 2s 39us/step - loss: 0.6959 - acc: 0.9388 - val_loss: 0.3081 - val_acc: 0.9382
Epoch 4/50
60000/60000 [==============================] - 2s 30us/step - loss: 0.6797 - acc: 0.9446 - val_loss: 0.3004 - val_acc: 0.9417
Epoch 5/50
60000/60000 [==============================] - 2s 28us/step - loss: 0.6695 - acc: 0.9486 - val_loss: 0.2861 - val_acc: 0.9449
Epoch 6/50
60000/60000 [==============================] - 2s 30us/step - loss: 0.6622 - acc: 0.9525 - val_loss: 0.2839 - val_acc: 0.9467
Epoch 7/50
60000/60000 [==============================] - 2s 31us/step - loss: 0.6564 - acc: 0.9557 - val_loss: 0.2758 - val_acc

### Student Net 3
- DISTILLED --- DENSE TEACHER + WEIGHTED HARD/SOFT TARGET

In [10]:
def avg_mix_loss(y_true, y_pred, w, T):    
    # split hard & soft targets
    y_hard, y_soft = y_true[:, :10], y_true[:, 10:]
    
    # convert logits to predicted values
    y_hard_pred = K.softmax(y_pred) # hard target
    y_soft_pred = K.softmax(y_pred / T) # soft target
    
    # compute weighted avg of the 2 parts of losses
    avg_loss = w * categorical_crossentropy(y_hard, y_hard_pred) + categorical_crossentropy(y_soft, y_soft_pred)
    
    return avg_loss

In [15]:
n_hidden = 20
T = 3
w = 0.7 / (T**2)

# apply both hard & soft targets to learn
logit_test = teacher_logit.predict(x_test)
y_test_soft = K.softmax(logit_test / T).eval(session=K.get_session())
y_hard_soft_train = np.concatenate((y_train, y_train_soft), axis=1)
y_hard_soft_test = np.concatenate((y_test, y_test_soft), axis=1)

# fit the student net distilled from the dense teacher net with the hard-soft weighted avg loss
mnist_student_mix = Sequential()
mnist_student_mix.add(Dense(n_hidden, name='hidden_1', input_shape=(28*28, ), activation='relu'))
mnist_student_mix.add(Dense(n_hidden, name='hidden_2', activation='relu'))
mnist_student_mix.add(Dense(10, name='logit'))

# y_pred at the end of 10-node dense layer is the logit_pred
mnist_student_mix.compile(loss=lambda y_true, y_pred: avg_mix_loss(y_true, y_pred, w, T), 
                          optimizer=Adam(), metrics=['accuracy'])

mnist_student_mix.fit(x_train, y_hard_soft_train, 
                      batch_size=100, epochs=50, verbose=1, validation_data=(x_test, y_hard_soft_test))


# distilled student net with mix-hard-soft loss model evaluation
loss, accuracy = mnist_student_mix.evaluate(x_test, y_hard_soft_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('DISTILLED STUDENT with WEIGHTED HARD/SOFT TARGET - On test set:')
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 3s 44us/step - loss: 1.0612 - acc: 0.8294 - val_loss: 0.7709 - val_acc: 0.9172
Epoch 2/50
60000/60000 [==============================] - 3s 43us/step - loss: 0.7508 - acc: 0.9234 - val_loss: 0.7099 - val_acc: 0.9344
Epoch 3/50
60000/60000 [==============================] - 3s 50us/step - loss: 0.7103 - acc: 0.9362 - val_loss: 0.6848 - val_acc: 0.9429
Epoch 4/50
60000/60000 [==============================] - 3s 43us/step - loss: 0.6903 - acc: 0.9442 - val_loss: 0.6726 - val_acc: 0.9472
Epoch 5/50
60000/60000 [==============================] - 2s 38us/step - loss: 0.6769 - acc: 0.9497 - val_loss: 0.6657 - val_acc: 0.9481
Epoch 6/50
60000/60000 [==============================] - 2s 36us/step - loss: 0.6675 - acc: 0.9533 - val_loss: 0.6559 - val_acc: 0.9543
Epoch 7/50
60000/60000 [==============================] - 2s 34us/step - loss: 0.6604 - acc: 0.9565 - val_loss: 0.6514 - val_acc

### Student Net 4
- DISTILLED --- CNN TEACHER

In [17]:
# compute 'soft target' of the CNN teacher net
T = 3
teacher_logit_cnn = get_layer_output(mnist_cnn, 'logit')
logit_train_cnn = teacher_logit_cnn.predict(x_train_2D)
y_train_soft_cnn = K.softmax(logit_train_cnn / T).eval(session=K.get_session())

# train student net distilled from the CNN teacher net
mnist_student_distilled_cnn = MNIST_StudentNet(n_hidden=20, T=T)
mnist_student_distilled_cnn.fit(x_train, y_train_soft_cnn, 
                                batch_size=128, epochs=50, verbose=1, validation_data=(x_test, y_test))

# student net model evalutation
loss, accuracy = mnist_student_distilled_cnn.evaluate(x_test, y_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('DISTILLED STUDENT from CNN TEACHER - On test set:')
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 3s 42us/step - loss: 0.9567 - acc: 0.7943 - val_loss: 0.3894 - val_acc: 0.9022
Epoch 2/50
60000/60000 [==============================] - 2s 37us/step - loss: 0.6094 - acc: 0.9093 - val_loss: 0.3409 - val_acc: 0.9193
Epoch 3/50
60000/60000 [==============================] - 2s 33us/step - loss: 0.5729 - acc: 0.9221 - val_loss: 0.3056 - val_acc: 0.9297
Epoch 4/50
60000/60000 [==============================] - 2s 33us/step - loss: 0.5469 - acc: 0.9314 - val_loss: 0.2815 - val_acc: 0.9345
Epoch 5/50
60000/60000 [==============================] - 2s 35us/step - loss: 0.5277 - acc: 0.9388 - val_loss: 0.2654 - val_acc: 0.9405
Epoch 6/50
60000/60000 [==============================] - 2s 35us/step - loss: 0.5138 - acc: 0.9443 - val_loss: 0.2596 - val_acc: 0.9394
Epoch 7/50
60000/60000 [==============================] - 3s 47us/step - loss: 0.5043 - acc: 0.9469 - val_loss: 0.2498 - val_acc

### Summary  performances of all distilled student nets

In [2]:
student_summary = {
    'baseline': {'loss':0.15354005151256278, 'accuracy': 0.9632, 'num_error': 368},
    'dense_distilled': {'loss': 0.23310526382923127, 'accuracy': 0.9654, 'num_error': 345},
    'dense_distilled_mix_loss': {'loss': 0.6234446800231933, 'accuracy': 0.9676, 'num_error': 323},
    'cnn_distilled': {'loss': 0.1805808451652527, 'accuracy': 0.966, 'num_error': 340}
}

print('=== Student Net Performance on Full Test Set ===')
df_student_summary = pd.DataFrame().from_dict(student_summary).T
display(df_student_summary)

=== Student Net Performance on Full Test Set ===


,accuracy,loss,num_error
baseline,0.9632,0.153540,368.0
cnn_distilled,0.9660,0.180581,340.0
dense_distilled,0.9654,0.233105,345.0
dense_distilled_mix_loss,0.9676,0.623445,323.0


---

## Experiment with omitting digit 3 in the transfer set

In [18]:
idx = (y_train[:,3] == 1)
x_train_omit3 = x_train[~idx,:]
y_train_omit3 = y_train[~idx,:]
y_train_soft_omit3 = y_train_soft[~idx,:]

In [19]:
# student net - NO distillation - digit 3 omitted in the transfer set
hard_omit3 = MNIST_StudentNet(n_hidden=20, T=1)
hard_omit3.fit(x_train_omit3, y_train_omit3, 
               batch_size=128, epochs=50, verbose=1, validation_data=(x_test, y_test))

Train on 53869 samples, validate on 10000 samples
Epoch 1/50
53869/53869 [==============================] - 3s 49us/step - loss: 0.6009 - acc: 0.8234 - val_loss: 1.2552 - val_acc: 0.8329
Epoch 2/50
53869/53869 [==============================] - 2s 31us/step - loss: 0.2178 - acc: 0.9372 - val_loss: 1.3667 - val_acc: 0.8483
Epoch 3/50
53869/53869 [==============================] - 2s 45us/step - loss: 0.1759 - acc: 0.9487 - val_loss: 1.4066 - val_acc: 0.8508
Epoch 4/50
53869/53869 [==============================] - 1s 25us/step - loss: 0.1539 - acc: 0.9545 - val_loss: 1.5036 - val_acc: 0.8594
Epoch 5/50
53869/53869 [==============================] - 2s 28us/step - loss: 0.1377 - acc: 0.9591 - val_loss: 1.5293 - val_acc: 0.8607
Epoch 6/50
53869/53869 [==============================] - 1s 26us/step - loss: 0.1264 - acc: 0.9620 - val_loss: 1.5749 - val_acc: 0.8619
Epoch 7/50
53869/53869 [==============================] - 1s 28us/step - loss: 0.1163 - acc: 0.9647 - val_loss: 1.5799 - val_acc

In [20]:
# student net - WITH distillation - digit 3 omitted in the transfer set
soft_omit3 = MNIST_StudentNet(n_hidden=20, T=3)
soft_omit3.fit(x_train_omit3, y_train_soft_omit3, 
               batch_size=128, epochs=50, verbose=1, validation_data=(x_test, y_test))

Train on 53869 samples, validate on 10000 samples
Epoch 1/50
53869/53869 [==============================] - 5s 89us/step - loss: 1.0650 - acc: 0.8091 - val_loss: 0.5897 - val_acc: 0.8284
Epoch 2/50
53869/53869 [==============================] - 2s 37us/step - loss: 0.7366 - acc: 0.9316 - val_loss: 0.5106 - val_acc: 0.8477
Epoch 3/50
53869/53869 [==============================] - 2s 41us/step - loss: 0.7044 - acc: 0.9451 - val_loss: 0.4765 - val_acc: 0.8564
Epoch 4/50
53869/53869 [==============================] - 2s 40us/step - loss: 0.6845 - acc: 0.9518 - val_loss: 0.4417 - val_acc: 0.8660
Epoch 5/50
53869/53869 [==============================] - 3s 47us/step - loss: 0.6720 - acc: 0.9569 - val_loss: 0.4382 - val_acc: 0.8674
Epoch 6/50
53869/53869 [==============================] - 2s 31us/step - loss: 0.6639 - acc: 0.9604 - val_loss: 0.4226 - val_acc: 0.8732
Epoch 7/50
53869/53869 [==============================] - 2s 38us/step - loss: 0.6581 - acc: 0.9627 - val_loss: 0.4026 - val_acc

In [32]:
n_hidden = 20
T = 5
w = 0.5 / (T**2) 

# apply both hard & soft targets to learn
y_hard_soft_train_omit3 = np.concatenate((y_train_omit3, y_train_soft_omit3), axis=1)

# student net - WITH distillation & weighted hard soft loss - digit 3 omitted in the transfer set
mix_omit3 = Sequential()
mix_omit3.add(Dense(n_hidden, name='hidden_1', input_shape=(28*28, ), activation='relu'))
mix_omit3.add(Dense(n_hidden, name='hidden_2', activation='relu'))
mix_omit3.add(Dense(10, name='logit'))

# y_pred at the end of 10-node dense layer is the logit_pred
mix_omit3.compile(loss=lambda y_true, y_pred: avg_mix_loss(y_true, y_pred, w, T), 
                  optimizer=Adam(), metrics=['accuracy'])

mix_omit3.fit(x_train_omit3, y_hard_soft_train_omit3, 
              batch_size=128, epochs=50, verbose=1, validation_data=(x_test, y_hard_soft_test))


Train on 53869 samples, validate on 10000 samples
Epoch 1/50
53869/53869 [==============================] - 3s 60us/step - loss: 1.1356 - acc: 0.8224 - val_loss: 1.0313 - val_acc: 0.8213
Epoch 2/50
53869/53869 [==============================] - 2s 32us/step - loss: 0.7748 - acc: 0.9208 - val_loss: 0.9304 - val_acc: 0.8357
Epoch 3/50
53869/53869 [==============================] - 2s 34us/step - loss: 0.7281 - acc: 0.9361 - val_loss: 0.8946 - val_acc: 0.8469
Epoch 4/50
53869/53869 [==============================] - 2s 34us/step - loss: 0.7045 - acc: 0.9451 - val_loss: 0.8583 - val_acc: 0.8546
Epoch 5/50
53869/53869 [==============================] - 2s 36us/step - loss: 0.6915 - acc: 0.9500 - val_loss: 0.8531 - val_acc: 0.8570
Epoch 6/50
53869/53869 [==============================] - 2s 29us/step - loss: 0.6825 - acc: 0.9539 - val_loss: 0.8302 - val_acc: 0.8626
Epoch 7/50
53869/53869 [==============================] - 2s 31us/step - loss: 0.6753 - acc: 0.9567 - val_loss: 0.8156 - val_acc

In [33]:
print('=== Overall Accuracy on Test set === \n')

loss, accuracy = hard_omit3.evaluate(x_test, y_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('NO DISTILLATION')
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))
print()

print('WITH DISTILLATION')
loss, accuracy = soft_omit3.evaluate(x_test, y_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))
print()

print('WITH DISTILLATION & WEIGHTED HARD-SOFT TARGET')
loss, accuracy = mix_omit3.evaluate(x_test, y_hard_soft_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))


=== Overall Accuracy on Test set === 

NO DISTILLATION
loss = 1.76703179179905, accuracy = 0.8678, #errors = 1321

WITH DISTILLATION
loss = 0.3264509074926376, accuracy = 0.9179, #errors = 820

WITH DISTILLATION & WEIGHTED HARD-SOFT TARGET
loss = 0.6823878009796143, accuracy = 0.93, #errors = 699


In [34]:
print('=== Digit-3 Only Accuracy on Test set === \n')

idx2 = y_test[:,3] == 1
x_test_3 = x_test[idx2, :]
y_test_3 = y_test[idx2, :]
y_test_soft_3 = y_test[idx2,:]
y_hard_soft_test_3 = np.concatenate((y_test_3, y_test_soft_3), axis=1)

loss, accuracy = hard_omit3.evaluate(x_test_3, y_test_3, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('NO DISTILLATION')
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))
print()

loss, accuracy = soft_omit3.evaluate(x_test_3, y_test_3, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('WITH DISTILLATION')
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))
print()

loss, accuracy = mix_omit3.evaluate(x_test_3, y_hard_soft_test_3, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('WITH DISTILLATION & WEIGHTED HARD-SOFT TARGET')
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))


=== Digit-3 Only Accuracy on Test set === 

NO DISTILLATION
loss = 16.111788532521466, accuracy = 0.0, #errors = 10000

WITH DISTILLATION
loss = 1.2265173893163699, accuracy = 0.4623762377417914, #errors = 5376

WITH DISTILLATION & WEIGHTED HARD-SOFT TARGET
loss = 1.1009060543362457, accuracy = 0.5722772281948882, #errors = 4277


In [36]:
# tune bias
soft_omit3_bias = MNIST_StudentNet(n_hidden=20, T=3)
bias = soft_omit3_bias.layers[2].get_weights()[1]
bias[3] = 3.5
K.set_value(soft_omit3_bias.layers[2].bias, bias)

soft_omit3_bias.fit(x_train_omit3, y_train_soft_omit3, 
                    batch_size=128, epochs=50, verbose=1, validation_data=(x_test, y_test))

Train on 53869 samples, validate on 10000 samples
Epoch 1/50
53869/53869 [==============================] - 3s 48us/step - loss: 1.0998 - acc: 0.7858 - val_loss: 0.6146 - val_acc: 0.8251
Epoch 2/50
53869/53869 [==============================] - 2s 34us/step - loss: 0.7507 - acc: 0.9262 - val_loss: 0.5272 - val_acc: 0.8416
Epoch 3/50
53869/53869 [==============================] - 2s 36us/step - loss: 0.7094 - acc: 0.9416 - val_loss: 0.4840 - val_acc: 0.8540
Epoch 4/50
53869/53869 [==============================] - 2s 38us/step - loss: 0.6870 - acc: 0.9500 - val_loss: 0.4469 - val_acc: 0.8631
Epoch 5/50
53869/53869 [==============================] - 2s 32us/step - loss: 0.6742 - acc: 0.9558 - val_loss: 0.4214 - val_acc: 0.8748
Epoch 6/50
53869/53869 [==============================] - 2s 38us/step - loss: 0.6658 - acc: 0.9598 - val_loss: 0.4051 - val_acc: 0.8854
Epoch 7/50
53869/53869 [==============================] - 2s 36us/step - loss: 0.6599 - acc: 0.9621 - val_loss: 0.3867 - val_acc

In [47]:
print('=== BIAS TUNED - Overall Accuracy on Test set === \n')
loss, accuracy = soft_omit3_bias.evaluate(x_test, y_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))
print()

print('=== BIAS TUNED - Digit-3 Only Accuracy on Test set === \n')
loss, accuracy = soft_omit3_bias.evaluate(x_test_3, y_test_3, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))


=== BIAS TUNED - Overall Accuracy on Test set === 

loss = 0.32033734719753265, accuracy = 0.9229, #errors = 770

=== BIAS TUNED - Digit-3 Only Accuracy on Test set === 

loss = 1.1231612833419649, accuracy = 0.5257425743459475, #errors = 4742


### Summary performances of distilled student nets which have NEVER seen digit 3

In [3]:
omit3_summary_overall = {
    'baseline': {'loss': 1.76703179179905, 'accuracy': 0.8678, 'num_error': 1321},
    'dense_distilled': {'loss': 0.3264509074926376, 'accuracy': 0.9179, 'num_error': 820},
    'dense_distilled_mix_loss': {'loss': 0.6823878009796143, 'accuracy': 0.93, 'num_error': 699},
    'bias_tuned': {'loss': 0.32033734719753265, 'accuracy': 0.9229, 'num_error': 770}
}
omit3_summary_3only = {
    'baseline': {'loss': 16.111788532521466, 'accuracy': 0.0, 'num_error': 10000},
    'dense_distilled': {'loss': 1.2265173893163699, 'accuracy': 0.4623762377417914, 'num_error': 5376},
    'dense_distilled_mix_loss': {'loss': 1.1009060543362457, 'accuracy': 0.5722772281948882, 'num_error': 4277},
    'bias_tuned': {'loss': 1.1231612833419649, 'accuracy': 0.5257425743459475, 'num_error': 4742}
}

df_omit3_summary_overall = pd.DataFrame().from_dict(omit3_summary_overall).T
df_omit3_summary_3only = pd.DataFrame().from_dict(omit3_summary_3only).T

print('=== Omit3 Distillation Performance on Full Test Set ===')
display(df_omit3_summary_overall)
print()

print('=== Omit3 Distillation Performance on Digit 3 Test Samples ===')
display(df_omit3_summary_3only)
print()

=== Omit3 Distillation Performance on Full Test Set ===


,accuracy,loss,num_error
baseline,0.8678,1.767032,1321.0
bias_tuned,0.9229,0.320337,770.0
dense_distilled,0.9179,0.326451,820.0
dense_distilled_mix_loss,0.9300,0.682388,699.0



=== Omit3 Distillation Performance on Digit 3 Test Samples ===


,accuracy,loss,num_error
baseline,0.000000,16.111789,10000.0
bias_tuned,0.525743,1.123161,4742.0
dense_distilled,0.462376,1.226517,5376.0
dense_distilled_mix_loss,0.572277,1.100906,4277.0


---

## Experiment with only keeping digit 7 and 8 in the transfer set

In [38]:
# keep 7 and 8 in transfer set
idx78 = [True if y[7]==1 or y[8] == 1 else False for y in y_train]

x_train_78 = x_train[idx78,:]
y_train_soft_78 = y_train_soft[idx78,:]
y_train_78 = y_train[idx78,:]

In [39]:
# train hard target model with training set omitting digit 3
hard_78 = MNIST_StudentNet(n_hidden=20, T=1)
hard_78.fit(x_train_78, y_train_78, batch_size=128, epochs=50, verbose=1, validation_data=(x_test, y_test))

Train on 12116 samples, validate on 10000 samples
Epoch 1/50
12116/12116 [==============================] - 2s 161us/step - loss: 0.3170 - acc: 0.9272 - val_loss: 8.2521 - val_acc: 0.1965
Epoch 2/50
12116/12116 [==============================] - 1s 58us/step - loss: 0.0342 - acc: 0.9889 - val_loss: 9.1068 - val_acc: 0.1973
Epoch 3/50
12116/12116 [==============================] - 1s 51us/step - loss: 0.0259 - acc: 0.9919 - val_loss: 9.5551 - val_acc: 0.1976
Epoch 4/50
12116/12116 [==============================] - 1s 48us/step - loss: 0.0220 - acc: 0.9931 - val_loss: 9.8298 - val_acc: 0.1974
Epoch 5/50
12116/12116 [==============================] - 1s 51us/step - loss: 0.0201 - acc: 0.9931 - val_loss: 10.0648 - val_acc: 0.1980
Epoch 6/50
12116/12116 [==============================] - 1s 53us/step - loss: 0.0183 - acc: 0.9941 - val_loss: 10.2347 - val_acc: 0.1979
Epoch 7/50
12116/12116 [==============================] - 1s 99us/step - loss: 0.0177 - acc: 0.9938 - val_loss: 10.3479 - val

In [40]:
# train soft target model with training set omitting digit 3
soft_78 = MNIST_StudentNet(n_hidden=20, T=3)
soft_78.fit(x_train_78, y_train_soft_78, batch_size=128, epochs=100, verbose=1, validation_data=(x_test, y_test))

Train on 12116 samples, validate on 10000 samples
Epoch 1/100
12116/12116 [==============================] - 2s 166us/step - loss: 1.3137 - acc: 0.8350 - val_loss: 2.7221 - val_acc: 0.1943
Epoch 2/100
12116/12116 [==============================] - 1s 67us/step - loss: 0.8349 - acc: 0.9833 - val_loss: 2.3942 - val_acc: 0.1964
Epoch 3/100
12116/12116 [==============================] - 1s 92us/step - loss: 0.7921 - acc: 0.9874 - val_loss: 2.2301 - val_acc: 0.1971
Epoch 4/100
12116/12116 [==============================] - 0s 40us/step - loss: 0.7810 - acc: 0.9888 - val_loss: 2.1232 - val_acc: 0.1986
Epoch 5/100
12116/12116 [==============================] - 0s 41us/step - loss: 0.7738 - acc: 0.9893 - val_loss: 2.0344 - val_acc: 0.2018
Epoch 6/100
12116/12116 [==============================] - 0s 40us/step - loss: 0.7676 - acc: 0.9901 - val_loss: 1.9403 - val_acc: 0.2127
Epoch 7/100
12116/12116 [==============================] - 0s 39us/step - loss: 0.7628 - acc: 0.9902 - val_loss: 1.8744 -

In [43]:
n_hidden = 20
T = 5
w = 0.5 / (T**2) 

# apply both hard & soft targets to learn
y_hard_soft_train_78 = np.concatenate((y_train_78, y_train_soft_78), axis=1)

# student net - WITH distillation & weighted hard soft loss - digit 3 omitted in the transfer set
mix_78 = Sequential()
mix_78.add(Dense(n_hidden, name='hidden_1', input_shape=(28*28, ), activation='relu'))
mix_78.add(Dense(n_hidden, name='hidden_2', activation='relu'))
mix_78.add(Dense(10, name='logit'))

# y_pred at the end of 10-node dense layer is the logit_pred
mix_78.compile(loss=lambda y_true, y_pred: avg_mix_loss(y_true, y_pred, w, T), 
               optimizer=Adam(), metrics=['accuracy'])

mix_78.fit(x_train_78, y_hard_soft_train_78, 
           batch_size=128, epochs=100, verbose=1, validation_data=(x_test, y_hard_soft_test))


Train on 12116 samples, validate on 10000 samples
Epoch 1/100
12116/12116 [==============================] - 4s 314us/step - loss: 1.4717 - acc: 0.8966 - val_loss: 3.1774 - val_acc: 0.1925
Epoch 2/100
12116/12116 [==============================] - 1s 110us/step - loss: 0.8514 - acc: 0.9808 - val_loss: 2.7144 - val_acc: 0.1955
Epoch 3/100
12116/12116 [==============================] - 1s 82us/step - loss: 0.8034 - acc: 0.9891 - val_loss: 2.5803 - val_acc: 0.1967
Epoch 4/100
12116/12116 [==============================] - 1s 81us/step - loss: 0.7909 - acc: 0.9908 - val_loss: 2.5080 - val_acc: 0.1974
Epoch 5/100
12116/12116 [==============================] - 1s 75us/step - loss: 0.7839 - acc: 0.9916 - val_loss: 2.4242 - val_acc: 0.1986
Epoch 6/100
12116/12116 [==============================] - 1s 67us/step - loss: 0.7786 - acc: 0.9922 - val_loss: 2.4065 - val_acc: 0.2003
Epoch 7/100
12116/12116 [==============================] - 1s 66us/step - loss: 0.7741 - acc: 0.9929 - val_loss: 2.3520 

In [44]:
soft_78_bias = MNIST_StudentNet(n_hidden=20, T=3)
bias = soft_78_bias.layers[2].get_weights()[1]
bias[7] = 1
bias[8] = 1
K.set_value(soft_78_bias.layers[2].bias, bias)

soft_78_bias.fit(x_train_78, y_train_soft_78, 
                 batch_size=128, epochs=100,verbose=1, validation_data=(x_test, y_test))

Train on 12116 samples, validate on 10000 samples
Epoch 1/100
12116/12116 [==============================] - 2s 158us/step - loss: 1.1857 - acc: 0.9439 - val_loss: 2.7235 - val_acc: 0.1951
Epoch 2/100
12116/12116 [==============================] - 1s 49us/step - loss: 0.8168 - acc: 0.9835 - val_loss: 2.4055 - val_acc: 0.1966
Epoch 3/100
12116/12116 [==============================] - 0s 40us/step - loss: 0.7902 - acc: 0.9879 - val_loss: 2.2500 - val_acc: 0.1972
Epoch 4/100
12116/12116 [==============================] - 1s 42us/step - loss: 0.7781 - acc: 0.9889 - val_loss: 2.1407 - val_acc: 0.1979
Epoch 5/100
12116/12116 [==============================] - 1s 43us/step - loss: 0.7688 - acc: 0.9896 - val_loss: 1.9956 - val_acc: 0.1990
Epoch 6/100
12116/12116 [==============================] - 1s 45us/step - loss: 0.7621 - acc: 0.9907 - val_loss: 1.9205 - val_acc: 0.2064
Epoch 7/100
12116/12116 [==============================] - 1s 49us/step - loss: 0.7574 - acc: 0.9910 - val_loss: 1.8677 -

In [48]:
print('=== Overall Accuracy on Test set === \n')

loss, accuracy = hard_78.evaluate(x_test, y_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('NO DISTILLATION')
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))
print()

print('WITH DISTILLATION')
loss, accuracy = soft_78.evaluate(x_test, y_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))
print()

print('WITH DISTILLATION & WEIGHTED HARD-SOFT TARGET')
loss, accuracy = mix_78.evaluate(x_test, y_hard_soft_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))
print()

print('WITH DISTILLATION & BIAS TUNED')
loss, accuracy = soft_78_bias.evaluate(x_test, y_test, verbose=0)
num_errors = int((1 - accuracy) * len(x_test))
print('loss = {}, accuracy = {}, #errors = {}'.format(loss, accuracy, num_errors))


=== Overall Accuracy on Test set === 

NO DISTILLATION
loss = 11.931765022277832, accuracy = 0.1986, #errors = 8014

WITH DISTILLATION
loss = 1.07515452003479, accuracy = 0.6182, #errors = 3818

WITH DISTILLATION & WEIGHTED HARD-SOFT TARGET
loss = 1.2918906003952026, accuracy = 0.6241, #errors = 3759

WITH DISTILLATION & BIAS TUNED
loss = 1.050957187271118, accuracy = 0.6302, #errors = 3698


### Summary performaces of distilled student nets which have ONLY SEEN digit 7 & 8

In [4]:
keep78_summary = {
    'baseline': {'loss': 11.931765022277832, 'accuracy': 0.1986, 'num_error': 8014},
    'dense_distilled': {'loss': 1.07515452003479, 'accuracy': 0.6182, 'num_error': 3818},
    'dense_distilled_mix_loss': {'loss': 1.2918906003952026, 'accuracy': 0.6241, 'num_error': 3759},
    'bias_tuned': {'loss': 1.050957187271118, 'accuracy': 0.6302, 'num_error': 3698}
}

print('=== Only 7 & 8 Distillation Performance on Full Test Set ===')
df_keep78_summary = pd.DataFrame().from_dict(keep78_summary).T
display(df_keep78_summary)

=== Only 7 & 8 Distillation Performance on Full Test Set ===


,accuracy,loss,num_error
baseline,0.1986,11.931765,8014.0
bias_tuned,0.6302,1.050957,3698.0
dense_distilled,0.6182,1.075155,3818.0
dense_distilled_mix_loss,0.6241,1.291891,3759.0
